In [70]:
import pandas as pd 
from pathlib import Path
import numpy as np
import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

In [71]:
# Read in the CSV file as a Pandas Dataframe
df = pd.read_csv(Path("resources/baseballdata.csv"))
df.set_index("last_name, first_name", inplace=True)
df.drop(columns=["player_id"], inplace=True)
print(df.columns)

Index(['year', 'player_age', 'ab', 'pa', 'hit', 'single', 'double', 'triple',
       'home_run', 'strikeout', 'walk', 'k_percent', 'bb_percent',
       'batting_avg', 'slg_percent', 'on_base_percent', 'on_base_plus_slg',
       'isolated_power', 'babip', 'woba', 'wobacon', 'bacon',
       'exit_velocity_avg', 'launch_angle_avg', 'sweet_spot_percent', 'barrel',
       'barrel_batted_rate', 'solidcontact_percent', 'flareburner_percent',
       'poorlyunder_percent', 'poorlytopped_percent', 'poorlyweak_percent',
       'hard_hit_percent', 'avg_best_speed', 'avg_hyper_speed',
       'z_swing_percent', 'z_swing_miss_percent', 'oz_swing_percent',
       'oz_swing_miss_percent', 'oz_contact_percent', 'out_zone_swing_miss',
       'out_zone_swing', 'out_zone_percent', 'out_zone',
       'meatball_swing_percent', 'iz_contact_percent', 'in_zone_swing_miss',
       'in_zone_swing', 'whiff_percent', 'swing_percent', 'pull_percent',
       'straightaway_percent', 'opposite_percent', 'batted_ball',


In [72]:
#Scale the Data
scaler = StandardScaler()
scaler.fit(df)

StandardScaler()

In [73]:
#Assign Variables to be Trained
X = df[["k_percent","bb_percent","solidcontact_percent","flareburner_percent","poorlyweak_percent","hard_hit_percent"]]
y = df["batting_avg"]

In [74]:
#Split the data into training & testing
X_train, X_test, y_train, y_test = train_test_split(X, y , test_size=0.2, random_state = 50)


In [75]:
#Create a Rabndom Forest Regressor Model
rf_model = RandomForestRegressor(n_estimators=100, random_state=50)

In [76]:
#Train the Model
rf_model.fit(X_train, y_train)

RandomForestRegressor(random_state=50)

In [77]:
#Make Predictions
predictions = rf_model.predict(X_test)
print(predictions)


[0.31097 0.26275 0.244   0.27464 0.2297  0.31004 0.26548 0.26504 0.23762
 0.2734  0.25447 0.26402 0.26269 0.26988 0.27279 0.24287 0.25725 0.29261
 0.2615  0.26214 0.25403 0.25868 0.30427 0.23963 0.27235 0.28399 0.27188
 0.24571 0.26315 0.25356 0.27173 0.24955 0.25696 0.26523 0.28524 0.2792
 0.23343 0.25628 0.28313 0.2437  0.2485  0.2586  0.25152 0.25642 0.26458
 0.24282 0.28186 0.24917 0.2659  0.2368  0.26967 0.23593 0.26862 0.26285
 0.24268 0.27338 0.26285 0.26234 0.23114 0.30275 0.27542 0.29209 0.24457
 0.28226 0.20651 0.26296 0.21699 0.24483 0.294   0.27404 0.26373 0.30361
 0.28658 0.26995 0.26177 0.29976 0.26369 0.3006  0.27195 0.29388 0.24729
 0.27041 0.2499  0.26327 0.28153 0.28035 0.30083 0.30676 0.2402  0.27144
 0.27264 0.27345 0.2909  0.27333 0.27228 0.25502 0.25166 0.2566  0.28439
 0.25049 0.25534 0.26771 0.22582 0.25518 0.21577 0.32942 0.27074 0.25268
 0.28742 0.21686 0.24127 0.25872 0.26635 0.28761 0.26592 0.24838 0.24997
 0.27083 0.25787 0.27306 0.24954 0.27461 0.28358 0.2

In [78]:
mean = mean_absolute_error(y_test, predictions)
print("Mean", mean)

Mean 0.02119445229681979
